In [23]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [19]:
# Setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # get training data
    download=True, # download data if it doesn't exist on disk
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


/Users/michael.todisco/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/torchvision/datasets/mnist.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484782168/work/torch/csrc/utils/tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


In [24]:
# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data, # dataset to turn into iterable
    batch_size=BATCH_SIZE, # how many samples per batch? 
    shuffle=True # shuffle data every epoch?
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False # don't necessarily have to shuffle the testing data
)

# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7fadcc730b50>, <torch.utils.data.dataloader.DataLoader object at 0x7fadcc7306a0>)
Length of train dataloader: 1875 batches of 32
Length of test dataloader: 313 batches of 32


In [92]:
class TorchVision(nn.Module):
    def __init__(self,
                 input_shape,
                 hidden_units,
                 output_shape,
                ):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                      out_channels=hidden_units,
                      kernel_size = 3,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size = 3,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*14*14, output_shape)
        )
        
    def forward(self, x):
        x = self.block_1(x)
        print(f'block 1 shape: {x.shape}')
        x = self.block_2(x)
        print(f'block 2 shape: {x.shape}')
        x = self.classifier(x)
        print(f'output shape: {x.shape}')
        return x

In [93]:
model = TorchVision(input_shape=1, hidden_units=10, output_shape=len(train_data.classes))

In [96]:
torch.manual_seed(42)

# Create sample batch of random numbers with same size as image batch
images = torch.randn(size=(32, 1, 64, 64)) # [batch_size, color_channels, height, width]
test_image = images[0]
model(test_image.unsqueeze(dim=0))

block 1 shape: torch.Size([1, 10, 31, 31])
block 2 shape: torch.Size([1, 10, 14, 14])
output shape: torch.Size([1, 10])


tensor([[-0.1284,  0.0748, -0.0469,  0.0891,  0.2372,  0.2351, -0.0720,  0.1312,
         -0.4682,  0.0789]], grad_fn=<AddmmBackward0>)

In [ ]:
def train_step()